In [2]:
from collections import deque, defaultdict
import random


class CityMap:
    def __init__(self, edges=None, num_nodes=50):
        self.map = defaultdict(list)

        if edges:
            for a, b in edges:
                self.map[a].append(b)
                self.map[b].append(a)
        else:
            self._generate_connected_map(num_nodes)

    def _generate_connected_map(self, n):
        for i in range(n - 1):
            self.map[i].append(i + 1)
            self.map[i + 1].append(i)

        extra_edges = int(n * 1.5)
        for _ in range(extra_edges):
            u = random.randint(0, n - 1)
            v = random.randint(0, n - 1)
            if u != v and v not in self.map[u]:
                self.map[u].append(v)
                self.map[v].append(u)

    def bfs(self, start, goal):
        q = deque([[start]])
        seen = {start}
        explored = 0

        while q:
            path = q.popleft()
            node = path[-1]
            explored += 1

            if node == goal:
                return path, explored

            for nxt in self.map[node]:
                if nxt not in seen:
                    seen.add(nxt)
                    q.append(path + [nxt])

        return None, explored

    def dfs(self, start, goal):
        stack = [[start]]
        seen = set()
        explored = 0

        while stack:
            path = stack.pop()
            node = path[-1]

            if node in seen:
                continue

            seen.add(node)
            explored += 1

            if node == goal:
                return path, explored

            for nxt in self.map[node]:
                if nxt not in seen:
                    stack.append(path + [nxt])

        return None, explored

    def bidirectional_bfs(self, start, goal):
        if start == goal:
            return [start], 0

        q1 = deque([start])
        q2 = deque([goal])

        from_start = {start: None}
        from_goal = {goal: None}

        explored = 0

        while q1 and q2:
            a = q1.popleft()
            explored += 1

            if a in from_goal:
                return self._merge_paths(from_start, from_goal, a), explored

            for n in self.map[a]:
                if n not in from_start:
                    from_start[n] = a
                    q1.append(n)

            b = q2.popleft()
            explored += 1

            if b in from_start:
                return self._merge_paths(from_start, from_goal, b), explored

            for n in self.map[b]:
                if n not in from_goal:
                    from_goal[n] = b
                    q2.append(n)

        return None, explored

    def _merge_paths(self, left_map, right_map, mid):
        left = []
        cur = mid
        while cur is not None:
            left.append(cur)
            cur = left_map[cur]
        left.reverse()

        right = []
        cur = right_map[mid]
        while cur is not None:
            right.append(cur)
            cur = right_map[cur]

        return left + right


if __name__ == "__main__":
    city = CityMap(num_nodes=100)

    start = 0
    end = 50

    bfs_path, bfs_count = city.bfs(start, end)
    dfs_path, dfs_count = city.dfs(start, end)
    bi_path, bi_count = city.bidirectional_bfs(start, end)

    print(f"{'Algorithm':<22} | {'Path Length':<12} | {'Nodes Explored':<15}")
    print("-" * 55)
    print(f"{'BFS':<22} | {len(bfs_path) if bfs_path else 'N/A':<12} | {bfs_count:<15}")
    print(f"{'DFS':<22} | {len(dfs_path) if dfs_path else 'N/A':<12} | {dfs_count:<15}")
    print(f"{'Bidirectional BFS':<22} | {len(bi_path) if bi_path else 'N/A':<12} | {bi_count:<15}")

    if bfs_path and bi_path:
        print("\nAnalysis:")
        print("Optimal length match:", len(bfs_path) == len(bi_path))
        print("Nodes saved by Bi-BFS:", bfs_count - bi_count)


Algorithm              | Path Length  | Nodes Explored 
-------------------------------------------------------
BFS                    | 2            | 4              
DFS                    | 28           | 28             
Bidirectional BFS      | 2            | 2              

Analysis:
Optimal length match: True
Nodes saved by Bi-BFS: 2
